In [1]:
!pip install pymongo


In [2]:
!pip install python-dotenv


In [3]:
from pymongo import MongoClient
from datetime import datetime

# Connect to local MongoDB (make sure MongoDB is running)
client = MongoClient("mongodb://localhost:27017/")
db = client["hospital_db"]

# Create collections
departments = db["departments"]
doctors = db["doctors"]
patients = db["patients"]
appointments = db["appointments"]
medical_records = db["medical_records"]

print("✅ Connected to MongoDB successfully!")


✅ Connected to MongoDB successfully!


In [4]:
from bson import ObjectId

def create_department(name, description=""):
    return departments.insert_one({
        "name": name,
        "description": description,
        "created_at": datetime.utcnow()
    }).inserted_id

def list_departments():
    return list(departments.find({}, {"name": 1, "description": 1}))

def create_doctor(name, department_id, specialization=""):
    return doctors.insert_one({
        "name": name,
        "department_id": ObjectId(department_id),
        "specialization": specialization,
        "created_at": datetime.utcnow()
    }).inserted_id

def list_doctors():
    docs = list(doctors.find())
    for d in docs:
        dept = departments.find_one({"_id": d["department_id"]})
        d["department_name"] = dept["name"] if dept else None
    return docs

def create_patient(name, dob, gender, phone):
    return patients.insert_one({
        "name": name,
        "dob": dob,
        "gender": gender,
        "phone": phone,
        "created_at": datetime.utcnow()
    }).inserted_id

def list_patients():
    return list(patients.find({}, {"name": 1, "gender": 1, "phone": 1}))

def create_appointment(patient_id, doctor_id, when, reason=""):
    return appointments.insert_one({
        "patient_id": ObjectId(patient_id),
        "doctor_id": ObjectId(doctor_id),
        "when": when,
        "reason": reason,
        "created_at": datetime.utcnow()
    }).inserted_id

def list_appointments():
    appts = list(appointments.find())
    for a in appts:
        pat = patients.find_one({"_id": a["patient_id"]})
        doc = doctors.find_one({"_id": a["doctor_id"]})
        a["patient_name"] = pat["name"] if pat else None
        a["doctor_name"] = doc["name"] if doc else None
    return appts


In [5]:
# Create departments
cardio_id = create_department("Cardiology", "Heart related treatments")
pediatrics_id = create_department("Pediatrics", "Child healthcare")

# Create doctors
doc1_id = create_doctor("Dr. Meena Sharma", cardio_id, "Cardiologist")
doc2_id = create_doctor("Dr. Rahul Patil", pediatrics_id, "Pediatrician")

# Create patients
pat1_id = create_patient("Amit Kumar", "1990-06-10", "M", "+911234567890")
pat2_id = create_patient("Neha Singh", "2015-05-15", "F", "+919876543210")

# Create appointment
appt_id = create_appointment(pat1_id, doc1_id, datetime(2025, 11, 5, 10, 30), "Chest pain")
print("✅ Sample data inserted successfully!")


C:\Users\Prithviraj\AppData\Local\Temp\ipykernel_12932\3644673965.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow()
C:\Users\Prithviraj\AppData\Local\Temp\ipykernel_12932\3644673965.py:18: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow()
C:\Users\Prithviraj\AppData\Local\Temp\ipykernel_12932\3644673965.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow()
C:\Users\Prithviraj\AppData\Local\Temp\ipykernel_12932\3644673965.py:46

✅ Sample data inserted successfully!


In [6]:
print("Departments:")
print(list_departments())

print("\nDoctors:")
print(list_doctors())

print("\nPatients:")
print(list_patients())

print("\nAppointments:")
print(list_appointments())


Departments:
[{'_id': ObjectId('6900e085f603f44d68c6e54f'), 'name': 'Cardiology', 'description': 'Heart related treatments'}, {'_id': ObjectId('6900e085f603f44d68c6e550'), 'name': 'Pediatrics', 'description': 'Child healthcare'}]

Doctors:
[{'_id': ObjectId('6900e085f603f44d68c6e551'), 'name': 'Dr. Meena Sharma', 'department_id': ObjectId('6900e085f603f44d68c6e54f'), 'specialization': 'Cardiologist', 'created_at': datetime.datetime(2025, 10, 28, 15, 25, 57, 508000), 'department_name': 'Cardiology'}, {'_id': ObjectId('6900e085f603f44d68c6e552'), 'name': 'Dr. Rahul Patil', 'department_id': ObjectId('6900e085f603f44d68c6e550'), 'specialization': 'Pediatrician', 'created_at': datetime.datetime(2025, 10, 28, 15, 25, 57, 530000), 'department_name': 'Pediatrics'}]

Patients:
[{'_id': ObjectId('6900e085f603f44d68c6e553'), 'name': 'Amit Kumar', 'gender': 'M', 'phone': '+911234567890'}, {'_id': ObjectId('6900e085f603f44d68c6e554'), 'name': 'Neha Singh', 'gender': 'F', 'phone': '+919876543210'}]
